## Gemini Model

A workflow that uses Google's Gemini LLM to search for novel and creative implementations of the NMF algorithm.

Initially will focus on improvements to the LS-NMF algorithm, then will search for improvements to the WS-NMF algorithm.

The workflow contains the following steps:
1. Prompt Gemini for a better implementation of the base algorithm, passing in the base model as text.
   1. The prompt will request n new models. 
2. Parse the results to extract the n 'new' versions of the algorithm.
3. Create a new python function for each of the algorithm.
4. Validate each of the new algorithms by creating a new BatchNMF model, and manually swapping out the self.update_step function with the new function.
5. Run 20 models, aggregate the results (mean runtime, min/mean/max Q(true), min/mean/max Q(robust))
6. Save the top 2 models to mongodb, along with aggregated results.
   1. Any number could be saved, but 2 provides options for the selection process.
7. Repeat steps 1-6 using a random selection from the new algorithm database.

In [1]:
import os
import sys
import pathlib
import textwrap
import inspect

import google.generativeai as genai

from IPython.display import display
from IPython.display import Markdown

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

C:\Users\dsmith\Anaconda3\envs\nmf_py\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
from src.data.datahandler import DataHandler
from src.model.nmf import NMF
from src.model.batch_nmf import BatchNMF

## Base LS-NMF update algorithm
from src.model.ls_nmf import LSNMF
base_alg = inspect.getsourcelines(LSNMF.update)
base_code = []
for i in range(len(base_alg[0])):
    if i > 0 and i < 7 or i >= 30:
        code_line = base_alg[0][i]
        if "def" in code_line:
            code_line = textwrap.dedent(code_line)
            if "self" not in code_line:
                code_line = code_line.replace("\n", "self,\n")
        base_code.append(code_line)        
# base_code = "".join(base_alg[0][1:7]) + "".join(base_alg[0][30:])
base_code = "".join(base_code)
# base_code = base_code[0: -1]
# base_code = str(base_code).replace("[", "")
# base_code = str(base_code).replace("]", "")
base_header = base_alg[0][1:7]
base_return = base_alg[0][-1:]
base_code

'def update(self,\n            V: np.ndarray,\n            We: np.ndarray,\n            W: np.ndarray,\n            H: np.ndarray\n    ):\n        WeV = np.multiply(We, V)\n        WH = np.matmul(W, H)\n        H_num = np.matmul(W.T, WeV)\n        H_den = np.matmul(W.T, np.multiply(We, WH))\n        H = np.multiply(H, np.divide(H_num, H_den))\n\n        WH = np.matmul(W, H)\n        W_num = np.matmul(WeV, H.T)\n        W_den = np.matmul(np.multiply(We, WH), H.T)\n        W = np.multiply(W, np.divide(W_num, W_den))\n\n        return W, H\n'

In [9]:
## Base WS-NMF update algorithm
from src.model.ws_nmf import WSNMF
ws_base_alg = inspect.getsourcelines(WSNMF.update)
ws_base_code = []
for i in range(1, len(ws_base_alg[0])):
    ws_code_line = ws_base_alg[0][i]
    if i < 7 or i > 33:
        if "def" in ws_code_line:
            ws_code_line = textwrap.dedent(ws_code_line)
            if "self" not in ws_code_line:
                ws_code_line = ws_code_line.replace("\n", "self,\n")
        ws_base_code.append(ws_code_line)        
        if "return" in ws_code_line:
            break
# ws_base_code = "".join(ws_base_code)
# ws_base_code = str(ws_base_code).replace("[", "")
# ws_base_code = str(ws_base_code).replace("]", "")
# ws_base_code = ws_base_code[0: -1]
ws_base_header = ws_base_alg[0][1:7]
ws_base_return = ws_base_alg[0][-1:]
ws_base_code

['def update(self,\n',
 '            V: np.ndarray,\n',
 '            We: np.ndarray,\n',
 '            W: np.ndarray,\n',
 '            H: np.ndarray\n',
 '    ):\n',
 '        uV = np.multiply(We, V)\n',
 '\n',
 '        _W = []\n',
 '        for i in range(V.shape[0]):\n',
 '            wei = We[i]\n',
 '            wei_d = np.diagflat(wei)\n',
 '\n',
 '            uv_i = uV[i]\n',
 '            uvi = uv_i.reshape(len(uv_i), 1)\n',
 '\n',
 '            _w_n = np.matmul(H, uvi).flatten()\n',
 '\n',
 '            uh = np.matmul(wei_d, H.T)\n',
 '            _w_d = np.matmul(H, uh)\n',
 '            _w_dm = np.matrix(_w_d)\n',
 '            if npl.det(_w_dm) == 0:\n',
 '                _w_di = np.array(npl.pinv(_w_dm))\n',
 '            else:\n',
 '                _w_di = np.array(npl.inv(_w_dm))\n',
 '            _w = np.matmul(_w_n, _w_di)\n',
 '            _W.append(_w)\n',
 '        W = np.array(_W)\n',
 '\n',
 '        W_n = (np.abs(W) - W) / 2.0\n',
 '        W_p = (np.abs(W) + W

In [4]:
import numpy as np
import json

output_directory = "D:\\projects\\nmf_py\\funsearch\\algorithms"

def run_algorithm(new_algorithm):
    input_file = os.path.join("data", "Dataset-BatonRouge-con.csv")
    uncertainty_file = os.path.join("data", "Dataset-BatonRouge-unc.csv")
    
    data_handler = DataHandler(
        input_path=input_file,
        uncertainty_path=uncertainty_file,
        index_col='Date'
    )
    V, U = data_handler.get_data()
    # TODO: In python code will change this to parallel=True (issues with Jupyter notebooks parallel method)
    nmf_models = BatchNMF(V=V, U=U, factors=6, models=10, method='ls-nmf', parallel=False, verbose=True)
    nmf_models.update_step = new_algorithm
    nmf_models.train()
    return nmf_models

def aggregate_results(models: BatchNMF):
    runtime = models.runtime / models.models
    qtrue = []
    qrobust = []
    for model in models.results:
        if model is None:
            continue
        qtrue.append(model.Qtrue)
        qrobust.append(model.Qrobust)
    return {"runtime": (round(runtime,2), round(models.runtime)), 
            "Q(true)": (round(np.min(qtrue),2), round(np.mean(qtrue),2), round(np.max(qtrue),2)), 
            "Q(robust)": (round(np.min(qrobust),2), round(np.mean(qrobust),2), round(np.max(qrobust),2))}

def update_summary(name, alg_summary, code_path):
    summary_file = os.path.join(output_directory, "summary.json")
    alg_summary["code_path"] = code_path
    if os.path.exists(summary_file):
        alg_key = name
        summary = {name: alg_summary}
        with open(summary_file, 'r') as sum_file:
            existing_summary = json.load(sum_file)
            if alg_key not in existing_summary.keys():
                existing_summary[alg_key] = alg_summary[alg_key]
            alg_summary = existing_summary
    else:
        summary = {name: alg_summary}
        alg_summary = summary
    with open(summary_file, "w") as sum_file:
        json.dump(alg_summary, sum_file)

def save_algorithm(name, code):
    code_file = os.path.join(output_directory, f"gemini-{name}.text")
    with open(code_file, "w") as cfile:
        for cline in code:
            cfile.write(cline)
    return code_file

def select_algorithm():
    summary_file = os.path.join(output_directory, "summary.json")
    index = 1
    if os.path.exists(summary_file):
        code_path = None
        alg_code = None
        with open(summary_file, 'r') as sum_file:
            existing_models = json.load(sum_file)
            model_keys = list(existing_models.keys())
            random_key = np.random.choice(model_keys,1)
            index = len(model_keys)
            while index in model_keys:
                index += 1
            code_path = existing_models[random_key]["code_path"]
        with open(code_path, 'r') as code_file:
            alg_code = code_file.read()
        return index, alg_code
    return index, None

In [5]:
base_models = run_algorithm(new_algorithm=ws_base_code)

27-Dec-23 08:32:48 - Input and output configured successfully
Model: 1, Seed: 8925, Q(true): 66257.34, Q(robust): 56038.48:  15%|██▎            | 3019/20000 [02:40<15:02, 18.83it/s]
Model: 2, Seed: 77395, Q(true): 68051.75, Q(robust): 58594.57:   8%|█             | 1590/20000 [01:25<16:31, 18.58it/s]
Model: 3, Seed: 65457, Q(true): 66606.33, Q(robust): 56599.35:  13%|█▊            | 2647/20000 [02:32<16:37, 17.40it/s]
Model: 4, Seed: 43887, Q(true): 65756.86, Q(robust): 56535.37:  10%|█▎            | 1947/20000 [01:57<18:06, 16.61it/s]
Model: 5, Seed: 43301, Q(true): 68131.03, Q(robust): 58718.47:   9%|█▎            | 1870/20000 [01:53<18:21, 16.46it/s]
Model: 6, Seed: 85859, Q(true): 67555.87, Q(robust): 57564.71:   8%|█             | 1510/20000 [01:33<19:02, 16.19it/s]
Model: 7, Seed: 8594, Q(true): 67871.81, Q(robust): 58203.47:   3%|▌               | 637/20000 [00:38<19:26, 16.59it/s]
Model: 8, Seed: 69736, Q(true): 66339.11, Q(robust): 56090.7:   7%|█              | 1434/20000 [01

In [ ]:
base_Qrobust = base_models.results[base_models.best_model].Qrobust
base_Qrobust

In [9]:
base_results = aggregate_results(models=base_models)
code_path = save_algorithm("0", base_code)
update_summary(0, base_results, code_path)

In [ ]:
# STEPS
# gemini_search = True
# n_algs = 4
# alg_keep = 2
# n_algs = 0
# added_algs = 0
# best_q = base_Qrobust
# best_alg = 0

# max_search = 500
# search_i = 0

# while gemini_search:
#     # Select a random existing algorithm
#     index, alg = select_algorithm()
#     if alg is None:
#         alg = base_code
#     # TODO: generate prompt
#     # TODO: parse prompt for model(s)
#     new_algs = []
#     alg_results = []
#     # for each model
#     for new_alg in new_algs:
#         try:
#             new_alg_result = run_algorithm(new_alg)
#         except Exception as e:
#             print(f"Algorithm failed due to error: {e}")
#         alg_qrobust = new_alg_result.results[new_alg_result.best_model].Qrobust
#         alg_results.append((new_alg, alg_qrobust, new_alg_result))
#         n_algs += 1
#     alg_results.sort(key=lambda a: a[2])
#     for i, alg_result in enumerate(alg_results): 
#         if i >= 2:
#             continue
#         if alg_result[1] > 2*base_Qrobust:
#             print(f"Algorithm failed due to Q(robust) being greater than 2*base_Qrobust. Model Q(Robust): {alg_result[1]}")
#             continue
#         elif alg_result[1] < base_Qrobust:
#             best_q = base_Qrobust
#             best_alg = index
#         # if the model succeeded, aggregate the results with 
#         results = aggregate_results(models=alg_result[2])
#         # write the model code to file
#         alg_file = save_algorithm(index, alg_result[0])
#         # write the results to the summary
#         update_summary(index, results, alg_file)
#         index += 1
#         added_algs
#     print(f"Search: {search_i}/{max_search}, Algorithms tested: {n_algs}, Algorithms added: {add_algs}, Current index: {index}, Base Q(robust): {base_Qrobust}, Best Q(robust): {best_q}, Best Algorithm: {best_alg}")
#     search_i += 1
#     if search_i > max_search:
#         gemini_search = False

In [4]:
GOOGLE_API_KEY = os.getenv("GOOGLE_GEMINI_KEY")
genai.configure(api_key=GOOGLE_API_KEY)
model = genai.GenerativeModel('gemini-pro')

In [5]:
chat = model.start_chat(history=[])

In [6]:
test_prompt = f"Can you give me an optimized version of the Non-Negative Matrix Factorization algorithm with input weights (We) using the provided input parameters and outputs in Python only using numpy, and return W and H? Function must be called update and take self as the first argument. The inputs have dimensions V: (NxM), We: (NxM), W: (Nxk) and H: (kxM). Here is the original to work from: {base_code}"

In [7]:
response = chat.send_message(test_prompt)

In [12]:
test = str(chat.history)
test

'[parts {\n  text: "Can you give me an optimized version of the Non-Negative Matrix Factorization algorithm with input weights (We) using the provided input parameters and outputs in Python only using numpy, and return W and H? Function must be called update and take self as the first argument. The inputs have dimensions V: (NxM), We: (NxM), W: (Nxk) and H: (kxM). Here is the original to work from: def update(self,\\n            V: np.ndarray,\\n            We: np.ndarray,\\n            W: np.ndarray,\\n            H: np.ndarray\\n    ):\\n        WeV = np.multiply(We, V)\\n        WH = np.matmul(W, H)\\n        H_num = np.matmul(W.T, WeV)\\n        H_den = np.matmul(W.T, np.multiply(We, WH))\\n        H = np.multiply(H, np.divide(H_num, H_den))\\n\\n        WH = np.matmul(W, H)\\n        W_num = np.matmul(WeV, H.T)\\n        W_den = np.matmul(np.multiply(We, WH), H.T)\\n        W = np.multiply(W, np.divide(W_num, W_den))\\n\\n        return W, H\\n"\n}\nrole: "user"\n, parts {\n  text

In [15]:
new_alg = response.text
new_alg

'```python\ndef update(self,\n            V: np.ndarray,\n            We: np.ndarray,\n            W: np.ndarray,\n            H: np.ndarray\n    ):\n        WeV = We * V\n        WH = np.dot(W, H)\n        H_num = np.dot(W.T, WeV)\n        H_den = np.dot(W.T, We * WH)\n        H *= H_num / H_den\n\n        WH = np.dot(W, H)\n        W_num = np.dot(WeV, H.T)\n        W_den = np.dot(We * WH, H.T)\n        W *= W_num / W_den\n\n        return W, H\n```'

In [ ]:
def parse_response(new_alg_str):
    alg_list = (new_alg_str).split("\n")
    new_alg = []
    copy = False
    for code_line in alg_list:
        if "def" in code_line:
            copy = True
        if copy:
            new_alg.append(code_line)
        if "return" in code_line:
            break
    return "\n".join(new_alg)

In [17]:
new_code = parse_response(new_alg_str=new_alg)
new_code

'def update(self,\n            V: np.ndarray,\n            We: np.ndarray,\n            W: np.ndarray,\n            H: np.ndarray\n    ):\n        WeV = We * V\n        WH = np.dot(W, H)\n        H_num = np.dot(W.T, WeV)\n        H_den = np.dot(W.T, We * WH)\n        H *= H_num / H_den\n\n        WH = np.dot(W, H)\n        W_num = np.dot(WeV, H.T)\n        W_den = np.dot(We * WH, H.T)\n        W *= W_num / W_den\n\n        return W, H'

In [ ]:
run_algorithm(new_code)

19-Dec-23 14:34:20 - Input and output configured successfully
Model: 1, Seed: 8925, Q(true): 65044.23, Q(robust): 55637.36:  20%|██▊           | 4097/20000 [00:17<01:06, 240.63it/s]
Model: 2, Seed: 77395, Q(true): 65540.67, Q(robust): 55585.32:   9%|█            | 1701/20000 [00:06<01:13, 248.87it/s]
Model: 3, Seed: 65457, Q(true): 65037.55, Q(robust): 55592.8:  12%|█▌            | 2314/20000 [00:09<01:11, 249.04it/s]
Model: 4, Seed: 43887, Q(true): 66037.44, Q(robust): 56530.11:  11%|█▍           | 2142/20000 [00:08<01:11, 250.74it/s]
Model: 5, Seed: 43301, Q(true): 63834.38, Q(robust): 54425.78:  15%|██           | 3077/20000 [00:12<01:07, 250.53it/s]
Model: 6, Seed: 85859, Q(true): 63846.58, Q(robust): 54431.89:  11%|█▍           | 2206/20000 [00:09<01:12, 246.22it/s]

In [ ]:
prompt1 = "Get really creative with the algorithm and try something novel? But don't change the signature"
response1 = chat.send_message(prompt1)

In [ ]:
new_code1 = parse_response(new_alg_str=response1.text)
new_code1

In [ ]:
run_algorithm(new_code1)

In [ ]:
prompt2 = "Give me a more creative algorithm"
response2 = chat.send_message(prompt2)

In [ ]:
new_code2 = parse_response(new_alg_str=response2.text)
response2.text

In [ ]:
run_algorithm(new_code2)

In [7]:
response_test = chat.send_message("Give me 20 prompts I can submit to you for optimizing, updating, being creative with a snippet of python code for matrix factorization.")
print(response_test.text)

1. Optimize a Python function that performs matrix factorization using the singular value decomposition (SVD) method to reduce its computational complexity.


2. Update a Python script that uses the old `numpy.linalg.svd()` function for SVD to use the new `scipy.linalg.svd()` function, which provides more options and is more efficient for large matrices.


3. Refactor a Python method that calculates the Frobenius norm of a matrix to improve its readability and maintainability.


4. Optimize a Python program that performs non-negative matrix factorization (NMF) using the multiplicative update rules to improve its convergence speed.


5. Update a Python script that uses the old `sklearn.decomposition.NMF` module for NMF to use the new `scipy.sparse.linalg.svds` function, which is more efficient for sparse matrices.


6. Refactor a Python method that calculates the trace of a matrix to make it more robust and efficient for large matrices.


7. Optimize a Python program that performs matri

In [8]:
prompts = response_test.text.split("\n")
prompts

['1. Optimize a Python function that performs matrix factorization using the singular value decomposition (SVD) method to reduce its computational complexity.',
 '',
 '',
 '2. Update a Python script that uses the old `numpy.linalg.svd()` function for SVD to use the new `scipy.linalg.svd()` function, which provides more options and is more efficient for large matrices.',
 '',
 '',
 '3. Refactor a Python method that calculates the Frobenius norm of a matrix to improve its readability and maintainability.',
 '',
 '',
 '4. Optimize a Python program that performs non-negative matrix factorization (NMF) using the multiplicative update rules to improve its convergence speed.',
 '',
 '',
 '5. Update a Python script that uses the old `sklearn.decomposition.NMF` module for NMF to use the new `scipy.sparse.linalg.svds` function, which is more efficient for sparse matrices.',
 '',
 '',
 '6. Refactor a Python method that calculates the trace of a matrix to make it more robust and efficient for larg